In [34]:
import pandas as pd
df = pd.read_csv("dataset1.csv")

df.head()

,customer_id,sesso,età,data_iscrizione,tipo_abbonamento,prezzo_abbonamento,ultima_presenza,media_presenze_sett,giorni_da_ultima_presenza,ha_rinnovato,churn
0,CUST0000,M,56,2023-07-29,trimestrale,80,2025-05-10,1.88,43,True,0
1,CUST0001,M,38,2025-01-06,trimestrale,80,2025-06-16,2.19,6,True,0
2,CUST0002,M,36,2024-05-28,trimestrale,80,2025-02-23,0.00,119,False,1
3,CUST0003,M,57,2023-08-06,mensile,30,2025-06-20,2.25,2,True,0
4,CUST0004,M,39,2024-11-04,trimestrale,80,2025-06-06,1.66,16,False,0


In [35]:
df['data_iscrizione'] = pd.to_datetime(df['data_iscrizione'])

df['anno_iscrizione'] = df['data_iscrizione'].dt.year
df['mese_iscrizione'] = df['data_iscrizione'].dt.month
# df['giorno_iscrizione'] = df['data_iscrizione'].dt.day

df.drop('data_iscrizione', axis=1, inplace=True)

In [36]:
df['ultima_presenza'] = pd.to_datetime(df['ultima_presenza'])

# df['anno_ultima_presenza'] = df['ultima_presenza'].dt.year
df['mese_ultima_presenza'] = df['ultima_presenza'].dt.month
# df['giorno_ultima_presenza'] = df['ultima_presenza'].dt.day

df.drop('ultima_presenza', axis=1, inplace=True)

In [37]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['tipo_abbonamento_encoder'] = le.fit_transform(df['tipo_abbonamento'])

df.drop('tipo_abbonamento', axis=1, inplace=True)
print(df.head())

  customer_id sesso  età  prezzo_abbonamento  media_presenze_sett  \
0    CUST0000     M   56                  80                 1.88   
1    CUST0001     M   38                  80                 2.19   
2    CUST0002     M   36                  80                 0.00   
3    CUST0003     M   57                  30                 2.25   
4    CUST0004     M   39                  80                 1.66   

   giorni_da_ultima_presenza  ha_rinnovato  churn  anno_iscrizione  \
0                         43          True      0             2023   
1                          6          True      0             2025   
2                        119         False      1             2024   
3                          2          True      0             2023   
4                         16         False      0             2024   

   mese_iscrizione  mese_ultima_presenza  tipo_abbonamento_encoder  
0                7                     5                         2  
1                1        

In [38]:
df_encoded = pd.get_dummies(df, columns=['sesso'])

In [39]:
X = df_encoded.drop(['churn', 'customer_id', 'ha_rinnovato', 'mese_ultima_presenza'], axis=1)
y = df_encoded['mese_ultima_presenza']

print(X.head())
print(y.head())

   età  prezzo_abbonamento  media_presenze_sett  giorni_da_ultima_presenza  \
0   56                  80                 1.88                         43   
1   38                  80                 2.19                          6   
2   36                  80                 0.00                        119   
3   57                  30                 2.25                          2   
4   39                  80                 1.66                         16   

   anno_iscrizione  mese_iscrizione  tipo_abbonamento_encoder  sesso_F  \
0             2023                7                         2    False   
1             2025                1                         2    False   
2             2024                5                         2    False   
3             2023                8                         1    False   
4             2024               11                         2    False   

   sesso_M  
0     True  
1     True  
2     True  
3     True  
4     True  
0    5
1

In [40]:
print(df_encoded.head())

  customer_id  età  prezzo_abbonamento  media_presenze_sett  \
0    CUST0000   56                  80                 1.88   
1    CUST0001   38                  80                 2.19   
2    CUST0002   36                  80                 0.00   
3    CUST0003   57                  30                 2.25   
4    CUST0004   39                  80                 1.66   

   giorni_da_ultima_presenza  ha_rinnovato  churn  anno_iscrizione  \
0                         43          True      0             2023   
1                          6          True      0             2025   
2                        119         False      1             2024   
3                          2          True      0             2023   
4                         16         False      0             2024   

   mese_iscrizione  mese_ultima_presenza  tipo_abbonamento_encoder  sesso_F  \
0                7                     5                         2    False   
1                1                     6  

In [41]:
print(df_encoded.head())

  customer_id  età  prezzo_abbonamento  media_presenze_sett  \
0    CUST0000   56                  80                 1.88   
1    CUST0001   38                  80                 2.19   
2    CUST0002   36                  80                 0.00   
3    CUST0003   57                  30                 2.25   
4    CUST0004   39                  80                 1.66   

   giorni_da_ultima_presenza  ha_rinnovato  churn  anno_iscrizione  \
0                         43          True      0             2023   
1                          6          True      0             2025   
2                        119         False      1             2024   
3                          2          True      0             2023   
4                         16         False      0             2024   

   mese_iscrizione  mese_ultima_presenza  tipo_abbonamento_encoder  sesso_F  \
0                7                     5                         2    False   
1                1                     6  

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
df_test = df.loc[X_test.index] 

In [44]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,         
    max_depth=None,          
    min_samples_split=2,      
    min_samples_leaf=1,       
    max_features='sqrt',      
    random_state=42,          
    n_jobs=-1                 
)

In [45]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [46]:
for i, pred in enumerate(y_pred):
    customer_id = df_test.iloc[i]['customer_id']
    
    valore_reale = df[df['customer_id'] == customer_id]['mese_ultima_presenza'].values[0]

    print(f"L'utente con id {customer_id} è stato predetto che abbandonera nel mese {pred}, il valore reale era {valore_reale}")

L'utente con id CUST0521 è stato predetto che abbandonera nel mese 3.08, il valore reale era 3
L'utente con id CUST0737 è stato predetto che abbandonera nel mese 6.0, il valore reale era 6
L'utente con id CUST0740 è stato predetto che abbandonera nel mese 5.0, il valore reale era 5
L'utente con id CUST0660 è stato predetto che abbandonera nel mese 4.91, il valore reale era 5
L'utente con id CUST0411 è stato predetto che abbandonera nel mese 3.0, il valore reale era 3
L'utente con id CUST0678 è stato predetto che abbandonera nel mese 5.92, il valore reale era 6
L'utente con id CUST0626 è stato predetto che abbandonera nel mese 5.05, il valore reale era 5
L'utente con id CUST0513 è stato predetto che abbandonera nel mese 5.15, il valore reale era 5
L'utente con id CUST0859 è stato predetto che abbandonera nel mese 5.98, il valore reale era 6
L'utente con id CUST0136 è stato predetto che abbandonera nel mese 5.99, il valore reale era 6
L'utente con id CUST0811 è stato predetto che abbando

In [47]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(mae)
print(rmse)
print(r2)


0.057899999999999924
0.10710275439968853
0.9897685919289622


In [48]:
dati = "età, prezzo_abbonamento, media_presenze_sett, giorni_da_ultima_presenza, anno_iscrizione, mese_iscrizione, tipo_abbonamento_encoder, sesso_F, sesso_M"
lista = dati.split(",")
importanza = model.feature_importances_
for i in range(len(lista)):
    print(f"il dato {lista[i]} e' stato utile per la classificazione al {importanza[i]*100:.2f}%")

il dato età e' stato utile per la classificazione al 1.74%
il dato  prezzo_abbonamento e' stato utile per la classificazione al 0.32%
il dato  media_presenze_sett e' stato utile per la classificazione al 29.18%
il dato  giorni_da_ultima_presenza e' stato utile per la classificazione al 66.40%
il dato  anno_iscrizione e' stato utile per la classificazione al 0.35%
il dato  mese_iscrizione e' stato utile per la classificazione al 1.14%
il dato  tipo_abbonamento_encoder e' stato utile per la classificazione al 0.38%
il dato  sesso_F e' stato utile per la classificazione al 0.24%
il dato  sesso_M e' stato utile per la classificazione al 0.25%


In [49]:
import numpy as np
valori = np.array([27, 80, 3, 3, 2023, 2, 0, 1, 0])
valori_2d = valori.reshape(1, -1)

# print(valori)
# valori = np.array(valori)
# valori_2d = valori.reshape(1, -1)

new_pred = model.predict(valori_2d)
# new_pred = model.predict(valori_2d)
print(new_pred)

[6.]


/home/matteo/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [50]:
import joblib

joblib.dump(model, "modello_ultima_presenza.pkl")
print("Modello salvato con successo in 'modello_churn.pkl'")

Modello salvato con successo in 'modello_churn.pkl'
